In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn import preprocessing
import matplotlib.pyplot as plt

## Load the expression matrix
path = r'C:\Users\gah02\Desktop'
file_name = 'RNA_data_filtered.xlsx'
meta_data = 'RNA_meta.xlsx'

df = pd.read_excel(os.path.join(path,file_name), header=0)
df.head()

,Gene Symbol,CI_S2,NSO_S7,CO_S5,NSO_S13,NSI_S4,CO_S1,CI_S8,SO_S12,NSO_S1,NSI_S4_r2,SI_S3,SI_S9,CI_S2_r2,NSI_S10,SO_S6,CO_S11,SO_S5,SI_S3_r2
0,Lypla1,24.079291,21.707172,19.146823,20.713394,22.830100,15.788513,21.560327,20.100511,22.414835,21.251448,20.001733,20.504920,20.497199,19.739790,16.646626,23.346224,20.348296,15.691971
1,Tcea1,20.608178,17.689526,17.553119,21.589008,18.616490,19.485892,20.107432,23.134017,17.636222,17.232498,18.283769,18.528110,18.444198,20.497093,20.353191,20.967001,19.237499,18.963778
2,Atp6v1h,89.044930,80.407380,71.815830,92.199590,82.218917,71.641021,67.806006,89.375852,65.106021,74.076962,80.808502,74.600515,68.327180,81.719651,73.084380,85.882891,66.863011,73.120099
3,Oprk1,0.325566,0.447862,0.751140,0.735890,0.439426,0.463114,0.598512,0.477523,0.666630,0.556856,0.425557,0.451710,0.465866,0.583443,0.450374,0.498046,0.657638,0.323438
4,Rb1cc1,10.671190,14.777233,14.812280,11.002914,7.711099,16.126949,14.764892,10.212320,16.596037,15.132290,8.833831,15.927279,16.228641,15.516275,12.979485,8.201201,17.258120,15.328656


In [5]:
print(df.shape)

(10001, 19)


In [8]:
## Filter out non-expressed genes
expr_df = df.loc[df.sum(axis=1) > 0, :]
print(expr_df.shape)

(8721, 19)


In [10]:
## Filter out lowly expressed genes
mask_low_vals = (expr_df > 0.3).sum(axis=1) > 2
expr_df = expr_df.loc[mask_low_vals, :]
print(expr_df.shape)

(6784, 19)


In [13]:
meta_df = pd.read_excel(os.path.join(path,meta_data), sep='\t').set_index('Sample Name')
print(meta_df.shape)
# re-order the index to make it the same with expr_df
meta_df.reindex(expr_df.columns)
meta_df

(18, 5)


,Condition,Label,Type,Date,Day
Sample Name,,,,,
CI_S2,CI,C,IN,180725,Day1
NSI_S4,NSI,NS,IN,180725,Day1
NSO_S1,NSO,NS,OUT,180725,Day1
SI_S3,SI,S,IN,180725,Day1
SO_S5,SO,S,OUT,180725,Day1
CO_S1,CO,C,OUT,180725,Day1
CI_S2_r2,CI,C,IN,180726,Day2
CO_S5,CO,C,OUT,180726,Day2
NSI_S4_r2,NSI,NS,IN,180726,Day2


In [15]:
## Index the data frame by 'Gene Symbol' and create a dictionary to re-label the sample names

C_dict = {'CI_S2': 'C_180725_IN', 'CO_S1': 'C_180725_OUT', 'CI_S2_r2': 'C_180726_IN', 'CO_S5': 'C_180726_OUT', 'CI_S8':'C_180727_IN', 'CO_S11': 'C_180727_OUT','NSI_S4': 'N_180725_IN', 'NSO_S1': 'N_180725_OUT', 'NSI_S4_r2': 'N_180726_IN', 'NSO_S7': 'N_180726_OUT', 'NSI_S10':'N_180727_IN', 'NSO_S13': 'N_180727_OUT', 'SI_S3': 'S_180725_IN', 'SO_S5': 'S_180725_OUT', 'SI_S3_r2': 'S_180726_IN', 'SO_S6': 'S_180726_OUT', 'SI_S9':'S_180727_IN', 'SO_S12': 'S_180727_OUT'}

names = []
for i in df.iloc[0] :
    for k,v in zip(C_dict.keys(),C_dict.values()) :
        if i == k :
            names.append(v)


s = pd.Series(names)


df_I = expr_df.set_index('Gene Symbol')
df_I.head()

,CI_S2,NSO_S7,CO_S5,NSO_S13,NSI_S4,CO_S1,CI_S8,SO_S12,NSO_S1,NSI_S4_r2,SI_S3,SI_S9,CI_S2_r2,NSI_S10,SO_S6,CO_S11,SO_S5,SI_S3_r2
Gene Symbol,,,,,,,,,,,,,,,,,,
Lypla1,24.079291,21.707172,19.146823,20.713394,22.830100,15.788513,21.560327,20.100511,22.414835,21.251448,20.001733,20.504920,20.497199,19.739790,16.646626,23.346224,20.348296,15.691971
Tcea1,20.608178,17.689526,17.553119,21.589008,18.616490,19.485892,20.107432,23.134017,17.636222,17.232498,18.283769,18.528110,18.444198,20.497093,20.353191,20.967001,19.237499,18.963778
Atp6v1h,89.044930,80.407380,71.815830,92.199590,82.218917,71.641021,67.806006,89.375852,65.106021,74.076962,80.808502,74.600515,68.327180,81.719651,73.084380,85.882891,66.863011,73.120099
Oprk1,0.325566,0.447862,0.751140,0.735890,0.439426,0.463114,0.598512,0.477523,0.666630,0.556856,0.425557,0.451710,0.465866,0.583443,0.450374,0.498046,0.657638,0.323438
Rb1cc1,10.671190,14.777233,14.812280,11.002914,7.711099,16.126949,14.764892,10.212320,16.596037,15.132290,8.833831,15.927279,16.228641,15.516275,12.979485,8.201201,17.258120,15.328656


In [17]:
## Rename the columns to make it easier to link the condition, date, and type (input or output)
df_I.columns = ['C_180725_IN',
 'N_180726_OUT',
 'C_180726_OUT',
 'N_180727_OUT',
 'N_180725_IN',
 'C_180725_OUT',
 'C_180727_IN',
 'S_180727_OUT',
 'N_180725_OUT',
 'N_180726_IN',
 'S_180725_IN',
 'S_180727_IN',
 'C_180726_IN',
 'N_180727_IN',
 'S_180726_OUT',
 'C_180727_OUT',
 'S_180725_OUT',
 'S_180726_IN']

df_I.head()


,C_180725_IN,N_180726_OUT,C_180726_OUT,N_180727_OUT,N_180725_IN,C_180725_OUT,C_180727_IN,S_180727_OUT,N_180725_OUT,N_180726_IN,S_180725_IN,S_180727_IN,C_180726_IN,N_180727_IN,S_180726_OUT,C_180727_OUT,S_180725_OUT,S_180726_IN
Gene Symbol,,,,,,,,,,,,,,,,,,
Lypla1,24.079291,21.707172,19.146823,20.713394,22.830100,15.788513,21.560327,20.100511,22.414835,21.251448,20.001733,20.504920,20.497199,19.739790,16.646626,23.346224,20.348296,15.691971
Tcea1,20.608178,17.689526,17.553119,21.589008,18.616490,19.485892,20.107432,23.134017,17.636222,17.232498,18.283769,18.528110,18.444198,20.497093,20.353191,20.967001,19.237499,18.963778
Atp6v1h,89.044930,80.407380,71.815830,92.199590,82.218917,71.641021,67.806006,89.375852,65.106021,74.076962,80.808502,74.600515,68.327180,81.719651,73.084380,85.882891,66.863011,73.120099
Oprk1,0.325566,0.447862,0.751140,0.735890,0.439426,0.463114,0.598512,0.477523,0.666630,0.556856,0.425557,0.451710,0.465866,0.583443,0.450374,0.498046,0.657638,0.323438
Rb1cc1,10.671190,14.777233,14.812280,11.002914,7.711099,16.126949,14.764892,10.212320,16.596037,15.132290,8.833831,15.927279,16.228641,15.516275,12.979485,8.201201,17.258120,15.328656


In [18]:
## Add 9 new columns which represent the ratio for each condition (C,N,S) on each date(180725,26,27) between input and output

df_I['C_180725'] = df_I['C_180725_IN']/df_I['C_180725_OUT']
df_I['N_180725'] = df_I['N_180725_IN']/df_I['N_180725_OUT']
df_I['S_180725'] = df_I['S_180725_IN']/df_I['S_180725_OUT']
df_I['C_180726'] = df_I['C_180726_IN']/df_I['C_180726_OUT']
df_I['N_180726'] = df_I['N_180726_IN']/df_I['N_180726_OUT']
df_I['S_180726'] = df_I['S_180726_IN']/df_I['S_180726_OUT']
df_I['C_180727'] = df_I['C_180727_IN']/df_I['C_180727_OUT']
df_I['N_180727'] = df_I['N_180727_IN']/df_I['N_180727_OUT']
df_I['S_180727'] = df_I['S_180727_IN']/df_I['S_180727_OUT']

df_I.head()

,C_180725_IN,N_180726_OUT,C_180726_OUT,N_180727_OUT,N_180725_IN,C_180725_OUT,C_180727_IN,S_180727_OUT,N_180725_OUT,N_180726_IN,...,S_180726_IN,C_180725,N_180725,S_180725,C_180726,N_180726,S_180726,C_180727,N_180727,S_180727
Gene Symbol,,,,,,,,,,,,,,,,,,,,,
Lypla1,24.079291,21.707172,19.146823,20.713394,22.830100,15.788513,21.560327,20.100511,22.414835,21.251448,...,15.691971,1.525115,1.018526,0.982968,1.070527,0.979006,0.942652,0.923504,0.952996,1.020119
Tcea1,20.608178,17.689526,17.553119,21.589008,18.616490,19.485892,20.107432,23.134017,17.636222,17.232498,...,18.963778,1.057595,1.055583,0.950423,1.050765,0.974164,0.931735,0.959004,0.949423,0.800903
Atp6v1h,89.044930,80.407380,71.815830,92.199590,82.218917,71.641021,67.806006,89.375852,65.106021,74.076962,...,73.120099,1.242932,1.262847,1.208568,0.951422,0.921271,1.000489,0.789517,0.886334,0.834683
Oprk1,0.325566,0.447862,0.751140,0.735890,0.439426,0.463114,0.598512,0.477523,0.666630,0.556856,...,0.323438,0.702993,0.659175,0.647099,0.620212,1.243365,0.718153,1.201720,0.792839,0.945944
Rb1cc1,10.671190,14.777233,14.812280,11.002914,7.711099,16.126949,14.764892,10.212320,16.596037,15.132290,...,15.328656,0.661699,0.464635,0.511865,1.095621,1.024027,1.180991,1.800333,1.410197,1.559614
